In [4]:
import os
import pandas as pd
import re

# Define the path to the results directory
results_dir = "results"

# Function to extract model name from filename
def extract_model_name(filename):
    match = re.search(r"experimental_design_results_(.*)\.csv", filename)
    if match:
        return match.group(1)
    else:
        return "Unknown Model"

# Function to read CSV files from a directory, add 'model' column, and select specific columns
def read_and_label_csvs(directory):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            try:
                df = pd.read_csv(filepath, encoding='utf-8')
            
                model_name = extract_model_name(filename)
                df['model'] = model_name  # Add model name as a column
                
                # Select specific columns
                df = df[['model', 'baseline', 'results']]
                
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dfs

# Read and label CSV files from the results directory
df = read_and_label_csvs(results_dir)
print(df)

[                     model                                           baseline  \
0   gpt-4o-mini-2024-07-18  o CEP do endereço QNM 07 CONJUNTO H LOTE 17 S/...   
1   gpt-4o-mini-2024-07-18  o CEP do endereço QI 13 LOTE 01 A 14 ALA 01 BO...   
2   gpt-4o-mini-2024-07-18  o CEP do endereço SHC/ SW CLSW 103 BL A S/N LO...   
3   gpt-4o-mini-2024-07-18  o CEP do endereço 06 CONJUNTO 05/06 LOTE 3-A S...   
4   gpt-4o-mini-2024-07-18  o CEP do endereço SHCS CR QD 516, BLOCO B 69 1...   
..                     ...                                                ...   
95  gpt-4o-mini-2024-07-18  o CEP do endereço QD 104 CJ 09 LT 02 RECANTO D...   
96  gpt-4o-mini-2024-07-18  o CEP do endereço QNB 13 LOTE 37 CASA 01 TAGUA...   
97  gpt-4o-mini-2024-07-18  o CEP do endereço CRS 502 BL C LJ 37 PARTE 677...   
98  gpt-4o-mini-2024-07-18  o CEP do endereço QUADRA 1 COMERCIO LOCAL S/N ...   
99  gpt-4o-mini-2024-07-18  o CEP do endereço QS 9 RUA 122 LOTE 08 LOJA 03...   

                          

In [5]:
import os
from dotenv import load_dotenv
import openai
from tqdm import tqdm
import time

# Specify the path to your .env file
dotenv_path = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.env"

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

# Access and store the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
model = 'gpt-4o-mini-2024-07-18'

In [15]:
import json

# Iterate over each row and make API call
for index, row in tqdm(enumerate(df), total=len(df)):
    if index % 100 == 0 and index != 0:
        print("min. pause...")
        time.sleep(60)
    baseline = row['baseline']
    result = row['results']
    temperature = 0.0
    comparison_promt = f"""Avalie as respostas abaixo de 0 a 10, considerando os seguintes critérios:

        *Qualidade da Resposta*: A resposta é clara, concisa e bem estruturada? Ela aborda diretamente a questão ou tarefa proposta?
        *Concordância*: A resposta faz sentido no contexto da pergunta ou instrução? Ela segue uma linha de raciocínio coerente?
        *Precisão/Acurácia*: A resposta apresenta informações corretas e verificáveis comparadas com o baseline? Ela evita informações falsas ou enganosas?

        Resposta Base (Baseline): {baseline}
        Resposta do Modelo: {result}

        Atribua uma nota de 0 a 10 para a comparação entre as respostas (Baseline e Modelo), justificando brevemente sua avaliação. Retorne a avaliação no formato JSON, com as chaves "quality", "agreement", "accuracy" e "justification".
        """
        
    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": comparison_promt}],
        temperature=temperature
    )
    generated_text = response.choices[0].message.content
    
    # Load the JSON string
    metadata_str = generated_text.strip()
    if metadata_str.startswith("```json") and metadata_str.endswith("```"):
        metadata_str = metadata_str.strip("```").strip()
    if metadata_str.startswith("json"):
        metadata_str = metadata_str[4:].lstrip()
    data = pd.DataFrame([json.loads(metadata_str)])
    print(data)
    
    break
    
# Save the updated DataFrame
#output_filename = f"results/experimental_design_results_{model_name}.csv"
#df.to_csv(output_filename, index=False)

#print(f"Completed. Results saved in {output_filename}")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:03<?, ?it/s]

```json
{
  "quality": 2,
  "agreement": 3,
  "accuracy": 4,
  "justification": "A resposta do modelo é repetitiva e não fornece informações específicas sobre os CEPs solicitados, o que compromete a clareza e a estrutura da resposta. Embora algumas respostas mencionem que não há dados suficientes, isso não é verdade, pois a baseline contém informações claras sobre os CEPs. A concordância é baixa, pois a resposta não aborda diretamente a questão proposta. A precisão é um pouco melhor em uma das respostas, mas a maioria não apresenta informações úteis, resultando em uma nota geral baixa."
}
```
{'quality': 2, 'agreement': 3, 'accuracy': 4, 'justification': 'A resposta do modelo é repetitiva e não fornece informações específicas sobre os CEPs solicitados, o que compromete a clareza e a estrutura da resposta. Embora algumas respostas mencionem que não há dados suficientes, isso não é verdade, pois a baseline contém informações claras sobre os CEPs. A concordância é baixa, pois a resposta n

In [17]:
# Load the JSON string
metadata_str = generated_text.strip()
if metadata_str.startswith("```json") and metadata_str.endswith("```"):
    metadata_str = metadata_str.strip("```").strip()
if metadata_str.startswith("json"):
    metadata_str = metadata_str[4:].lstrip()
data = pd.DataFrame([json.loads(metadata_str)])
print(data)

   quality  agreement  accuracy  \
0        2          3         4   

                                       justification  
0  A resposta do modelo é repetitiva e não fornec...  


In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import json

# Iterate over each row and make API call
output_filename = f"model scoring.csv"
final_df['evaluation'] = None

# Initialize progress bar
progress_bar = IntProgress(min=0, max=len(final_df), desc=f"Processing {model}")
display(progress_bar)

for index, row in final_df.iterrows():
    if index % 100 == 0 and index != 0:
        print("min. pause...")
        time.sleep(60)
    try:
        tmp_comparison_prompt = row['comparison_prompt']

        response = openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": tmp_comparison_prompt}],
            temperature=float(row['temperature']),
            top_p=float(row['top_p'])
        )

        # Extract and store the generated text
        metadata_str = response.choices[0].message.content.strip()
        if metadata_str.startswith("```json") and metadata_str.endswith("```"):
            metadata_str = metadata_str.strip("```").strip()
        if metadata_str.startswith("json"):
            metadata_str = metadata_str[4:].lstrip()
        metadata = json.loads(metadata_str)

        filters = {}
        if isinstance(metadata, dict):
            filters = {k: v for k, v in metadata.items() if v is not None}

        final_df.loc[index, 'evaluation'] = filters

    except openai.OpenAIError as e:
        print(f"Error processing row {index} for model {model}: {e}")
        final_df.loc[index, 'evaluation'] = f"Error: {e}"  # Store the error message
    except Exception as e:
        print(f"Unexpected error processing row {index} for model {model}: {e}")
        final_df.loc[index, 'evaluation'] = f"Error: {e}"

    # Update progress bar
    progress_bar.value += 1

# Save the updated DataFrame (optional)
final_df.to_csv(output_filename, index=False)